In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [47]:
data = pd.read_csv('data\\KNN\\train.csv')

In [3]:
data.head(10)

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949
5,5,3.8099,1.9586,75,178065,6289802927
6,6,6.3336,4.3720,13,666829,9931249544
7,7,5.7409,6.7697,85,369002,5662813655
8,8,4.3114,6.9410,3,166384,8471780938
9,9,6.3414,0.0758,65,400060,1253803156


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
row_id      int64
x           float64
y           float64
accuracy    int64
time        int64
place_id    int64
dtypes: float64(2), int64(4)
memory usage: 1.3 GB


## 处理数据

### 缩小数据

In [48]:
data = data.query("x>1.0 & x<1.25 & y>2.5 & y<2.75").copy()

### 处理时间

In [49]:
time_list = pd.to_datetime(data['time'].values, unit='s')


In [29]:
time_list.hour

Int64Index([18,  2, 15, 23, 11, 16, 15, 10, 15, 23,
            ...
            10, 11,  8, 15,  1, 10, 23, 15,  0, 23],
           dtype='int64', length=17710)

### 构造一些特征

In [50]:
data['day'] = time_list.day
data['hour'] = time_list.hour
data['weekday'] = time_list.weekday


In [51]:
data.drop('time',axis=1,inplace=True)

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17710 entries, 600 to 29112154
Data columns (total 8 columns):
row_id      17710 non-null int64
x           17710 non-null float64
y           17710 non-null float64
accuracy    17710 non-null int64
place_id    17710 non-null int64
day         17710 non-null int64
hour        17710 non-null int64
weekday     17710 non-null int64
dtypes: float64(2), int64(6)
memory usage: 1.2 MB


### 删除签到数过少的数据

In [53]:
checkin_times=data.groupby('place_id')['row_id'].count()

In [54]:
checkin_times[checkin_times>3].index

Int64Index([1097200869, 1228935308, 1267801529, 1278040507, 1285051622,
            1325949552, 1435128522, 1446331025, 1459761417, 1479000473,
            ...
            9598377925, 9632980559, 9671190585, 9710472407, 9713454201,
            9741307878, 9753855529, 9806043737, 9809476069, 9980711012],
           dtype='int64', name='place_id', length=239)

In [57]:
data = data[data['place_id'].isin(checkin_times[checkin_times>3].index)]

### 取出特征值和目标值

In [60]:
y = data['place_id']
x = data.drop('place_id', axis=1)


### 进行数据分割

In [64]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

### 特征工程

In [77]:
#把 X_train 和 x_test 进行标准化
std= StandardScaler()
x_train = std.fit_transform(x_train)
x_test = std.transform(x_test) #此处不用再fit了 why?


C:\miniconda3\envs\ml_py35\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\miniconda3\envs\ml_py35\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\miniconda3\envs\ml_py35\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


### 算法流程

In [69]:
KNN = KNeighborsClassifier(n_neighbors=5)

In [78]:
# fit predict score
KNN.fit(x_train,y_train)
y_predict = KNN.predict(x_test)

In [79]:
KNN.score(x_test,y_test)

0.39527186761229316